### Take a list of images (jpg) + yolo labels and apply Albumentations augmentations

to generate new images correctly labeles (with BB in Yolo v5 format)

In [1]:
from os import path
import glob
import cv2
import albumentations as A
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from common_functions import read_bb, write_bb, yolo_to_cv2, do_check

In [3]:
# here we have to define a working way taking into account how the old_name structure is
# in this test the files are from Roboflow, so there are lots of "."
# you can find another way
def new_name(orig_name):
    # taking the first split.. it is one of the possible choices
    new_name_without_ext = orig_name.split(".")[0]

    return new_name_without_ext

In [4]:
DATASET_BASE_DIR = "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/train"
# under this dir I have images and labels

IMAGE_DEST_DIR = (
    "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images"
)
LABELS_DEST_DIR = (
    "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels"
)

images_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "images/*.jpg")))
labels_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "labels/*.txt")))

In [8]:
# process in loop
HOW_MANY_NEW_IMAGES = 5

for img_path_name, labels_path_name in zip(images_list, labels_list):
    # remove path
    img_only_name = img_path_name.split("/")[-1]
    labels_only_name = labels_path_name.split("/")[-1]

    print(f"Processing {img_only_name}, {labels_only_name}")

    for i in range(HOW_MANY_NEW_IMAGES):
        # every time Albumentations generate a new img + bb
        # build file names
        i += 1
        new_name_jpg = new_name(img_only_name) + f"_{i}.jpg"
        new_name_txt = new_name(img_only_name) + f"_{i}.txt"

        print(f"Saving {new_name_jpg} in {IMAGE_DEST_DIR}")
        print(f"Saving {new_name_txt} in {LABELS_DEST_DIR}")

    print()
    print()

Processing 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.jpg, 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.txt
Saving 00222_clear_jpg_1.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_1.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_2.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_2.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_3.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_3.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_4.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_4.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_t